Demo to Rutvik Dhopate

In [6]:
import numpy as np
file_path =" https://www.ccs.neu.edu/home/vip/teach/DMcourse/2_cluster_EM_mixt/HW2/coin_flips_outcome.txt "

def load_data(filename):
    return np.loadtxt(filename)

df = load_data(file_path)

pi: Probabilities of coin[A,B,C] is used at the experiment
p = Probabilities of coin[A,B,C] gives Head

In [16]:
import numpy as np
from scipy.special import comb

def binomial_prob(x, p, D):
    return comb(D, x) * (p ** x) * ((1 - p) ** (D - x))

def em_coin_flips(data, D, max_iter=100, tol=1e-6):
    N = len(data)
    K = 3  

    #initialize
    pi = np.random.dirichlet([1, 1, 1])  # Random initial pi_A, pi_B, pi_C
    p = np.random.rand(K)  # Random initial p_A, p_B, p_C

    for iteration in range(max_iter):
        # E-step: Compute responsibilities (r[n, k])
        responsibilities = np.zeros((N, K))
        for n in range(N):
            for k in range(K):
                responsibilities[n, k] = pi[k] * binomial_prob(data[n], p[k], D)
            responsibilities[n, :] /= np.sum(responsibilities[n, :])  # Normalize

        # M-step: Update pi and p
        N_k = responsibilities.sum(axis=0)  # Effective number of sessions for each coin
        pi = N_k / N  # Update pi
        p = np.array([
            np.sum(responsibilities[:, k] * data) / (N_k[k] * D) for k in range(K)
        ])  # Update p

        # log likelihood change
        log_likelihood = np.sum([
            np.log(np.sum([
                pi[k] * binomial_prob(data[n], p[k], D) for k in range(K)
            ])) for n in range(N)
        ])
        if iteration > 0 and abs(log_likelihood - prev_log_likelihood) < tol:
            break
        prev_log_likelihood = log_likelihood

    return pi, p

# Example data: Number of heads in each session (N=50, D=20)
data = np.array(df).sum(axis=1)
len(data)
# Run EM algorithm
D = 20  # Number of flips per session
pi, p = em_coin_flips(data, D)
print("Estimated pi:", pi)
print("Estimated p:", p)

Estimated pi: [0.51458877 0.30685623 0.178555  ]
Estimated p: [0.61005641 0.23694122 0.93172936]
